# Atualizar
- Adicionar persitencia de dados carregados, ou seja, implementar sistema de armazenamento dos vetores ex: chromadb
- organizar prompt 



In [4]:
from llama_index.llms.mistralai import MistralAI
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from dotenv import load_dotenv
import gradio as gr 
from os import path,getenv

In [5]:
load_dotenv()
api_key = getenv("api_key")

In [6]:
class ConfigChatIndex:
    def __init__(self, input_dir="data"):

        if not path.exists(input_dir):
            raise ValueError(f"O diretório {input_dir} não existe")

        self.input_dir = input_dir
        self.service_context = None
        self.index = None

    def initialize_settings(self):
        """Initialize settings for embedding, and node parsing."""
        Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
        Settings.node_parser = SentenceSplitter(chunk_size=500, chunk_overlap=50)
        Settings.num_output = 512
        Settings.context_window = 3900

    def build_index(self):
        """Build the VectorStoreIndex from the input directory."""
        data = SimpleDirectoryReader(input_dir=self.input_dir, recursive=True).load_data()
        self.index = VectorStoreIndex.from_documents(
            data,
            show_progress=True
        )
    def execute(self):
        try:
            self.initialize_settings()
            self.build_index()

            return self.index
        except Exception as e:
            raise Exception(f"Erro ao inicializar o chatbot: {str(e)}")

config_index = ConfigChatIndex( input_dir="data")  
data = config_index.execute()

Generating embeddings: 100%|██████████| 87/87 [01:02<00:00,  1.39it/s]


In [7]:
class Chatbot:
    def __init__(self, api_key, prompt, data ):

        self.api_key = api_key
        self.prompt = prompt
        self.service_context = None
        self.index = data
        self.chat_engine = None
        self.model_name = "HuggingFaceH4/zephyr-7b-alpha"

        try:
            self.initialize_llm()
        except Exception as e:
            raise Exception(f"Erro ao inicializar o chatbot: {str(e)}")
        
    def initialize_llm(self):
        """Initialize settings for LLM, embedding, and node parsing."""
        Settings.llm = MistralAI(api_key=self.api_key, system_prompt=self.prompt, temperature=0.9)
        #Settings.llm = HuggingFaceLLM(model_name=self.model_name)
        self.chat_engine = self.index.as_chat_engine(
            chat_mode="condense_question",
            verbose=True)
    
        
    def respond(self, message, history = None):
        try:
            if not self.chat_engine:
                raise ValueError("Chat engine is not initialized.")
            response = self.chat_engine.chat(message)
            return response.response
        except Exception as e:
            print(f"Erro ao processar mensagem: {str(e)}")
            return "Desculpe, ocorreu um erro ao processar sua solicitação."

prompt = (
    "Responda apenas em Português"
    "Quando te derem alguma saudação, ou perguntgar sobre quem é, você deve se apresentar"
    "Você é um assistente que responde a perguntas sobre relatórios de investimentos de empresas"
    "Responda de forma clara e direta às perguntas feitas, com base nos documentos, indicando o tempo, periodo ou data de cada resultado."
    )

load_dotenv()
chatbot_instance = Chatbot(api_key=api_key, prompt=prompt, data=data)


In [8]:
def chatbot_interface(message,historico=None):
    try:
        response = chatbot_instance.respond(message, historico)
        return response
    except Exception as e:
        return f"Desculpe, ocorreu um erro ao processar sua solicitação: {str(e)}"

demo = gr.ChatInterface(
    fn=chatbot_interface,
    type="messages", 
    examples=["hello", "hola", "merhaba", "oi"], 
    title="Chatbot de Investimentos",
)


#demo = gr.Interface(fn=chatbot_interface, 
#                    inputs="text", 
#                    outputs="text", 
#                    title="Chatbot de Investimentos", 
#                    description="Chatbot de Investimentos para responder perguntas sobre relatórios de investimentos de empresas")
#
demo.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
